In [1]:
! pip install redis

   ---------------------------------------- 0.0/255.6 kB ? eta -:--:--
   - -------------------------------------- 10.2/255.6 kB ? eta -:--:--
   ------ -------------------------------- 41.0/255.6 kB 653.6 kB/s eta 0:00:01
   --------------- ---------------------- 102.4/255.6 kB 980.4 kB/s eta 0:00:01
   ------------------------------------ --- 235.5/255.6 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 255.6/255.6 kB 1.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


CREATING CONNECTION TO REDIS

In [13]:
import redis
import json
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Configuration
redis_host = os.getenv("REDIS_HOST")
redis_port = int(os.getenv("REDIS_PORT"))
redis_password = os.getenv("REDIS_PASSWORD")


# Initialize Redis client
r = redis.StrictRedis(host=redis_host, port=redis_port, password=redis_password, decode_responses=True)

# Test Connection
try:
    r.ping()
    print("Connected to Redis successfully!")
except redis.ConnectionError as e:
    print(f"Failed to connect to Redis: {e}")


Connected to Redis successfully!


CREATING A SESSION

In [14]:
def create_session(session_id, user_data, expiration_seconds=3600):
    """
    Create a new session with a unique session_id and user_data.
    :param session_id: Unique session identifier
    :param user_data: Dictionary containing user-specific data
    :param expiration_seconds: Session expiration time in seconds (default: 1 hour)
    """
    r.setex(session_id, expiration_seconds, json.dumps(user_data))
    print(f"Session {session_id} created with expiration in {expiration_seconds} seconds.")

# Example Usage
user_data = {"user_id": "U001", "name": "Alice", "preferences": {"theme": "dark"}}
create_session("session_123", user_data)


Session session_123 created with expiration in 3600 seconds.


RETRIEVE THE SESSION

In [16]:
def get_session_data(session_id):
    """
    Retrieve user data from a session by session_id.
    :param session_id: Unique session identifier
    :return: Dictionary containing user data or None if the session does not exist
    """
    session_data = r.get(session_id)
    if session_data:
        return json.loads(session_data)
    else:
        print(f"Session {session_id} does not exist.")
        return None

# Example Usage
session_data = get_session_data("session_123")
print("Retrieved Session Data:", session_data)


Retrieved Session Data: {'user_id': 'U001', 'name': 'Alice', 'preferences': {'theme': 'dark'}}


UPDATE SESSION

In [17]:
def update_session_data(session_id, new_user_data):
    """
    Update existing session data.
    :param session_id: Unique session identifier
    :param new_user_data: Dictionary containing updated user-specific data
    """
    if r.exists(session_id):
        r.set(session_id, json.dumps(new_user_data))
        print(f"Session {session_id} updated.")
    else:
        print(f"Session {session_id} does not exist.")

# Example Usage
updated_user_data = {"user_id": "U001", "name": "Alice", "preferences": {"theme": "light"}}
update_session_data("session_123", updated_user_data)


Session session_123 updated.


DELETE SESSION

In [18]:
def delete_session(session_id):
    """
    Delete a session by session_id.
    :param session_id: Unique session identifier
    """
    if r.delete(session_id):
        print(f"Session {session_id} deleted.")
    else:
        print(f"Session {session_id} does not exist.")

# Example Usage
delete_session("session_123")


Session session_123 deleted.
